In [1]:
# коллабные импорты
from google.colab import drive
from google.colab import files

In [2]:
drive.mount('/GD',force_remount=True)

Mounted at /GD


In [3]:
!cp -r /GD/'My Drive'/pics.zip .
!cp -r /GD/'My Drive'/my_metric.py .
!cp -r /GD/'My Drive'/detection_net.h5 .

!mkdir masks_predict

In [ ]:
!unzip pics.zip

In [5]:
import math
import sys
import os
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2

from my_metric import iou

from PIL import Image

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

In [6]:
filenames = os.listdir('pics')

In [7]:
# Размер изображений
Y = 480
X = 640

In [8]:
data_gen_args = dict(rescale=1./255,)

flow_args = dict(classes=[''],
                 target_size = (Y, X),
                 batch_size = 8,
                 class_mode=None,
                 shuffle=False,)

pics_datagen = ImageDataGenerator(**data_gen_args)
    
pics_datagen = pics_datagen.flow_from_directory('pics', **flow_args)

Found 3875 images belonging to 1 classes.


In [9]:
model = tf.keras.models.load_model('detection_net.h5', compile=False)

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.Adam(learning_rate=0.001),
              metrics=[iou])

In [ ]:
for pics_name in tqdm(filenames):

    image = Image.open('pics/' + pics_name)

    size = image.size

    image = np.array(image) / 255.

    pics_resize = cv2.resize(image,
                             (X, Y),
                             interpolation=cv2.INTER_NEAREST)
    
    pics_resize = pics_resize.reshape(-1, 480, 640, 3)
    
    pred_proba = model.predict(pics_resize)

    pred = ((pred_proba > 0.5).astype(int) * 255).reshape(480, 640, 1)

    mask_resize = cv2.resize(pred,
                             size,
                             interpolation=cv2.INTER_NEAREST)
    
    mask_name = pics_name.split('.')[0] + '_mask.jpg'
    
    cv2.imwrite('masks_predict/' + mask_name, mask_resize)


In [11]:
shutil.make_archive('masks_predict', 'zip', 'masks_predict')

'/content/masks_predict.zip'

In [12]:
!cp masks_predict.zip /GD/'My Drive'/masks_predict.zip